# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [13]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
import re

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
# load data from database
engine = create_engine('sqlite:///disaster_responses.db')
df = pd.read_sql_table('responses', engine)

X = df['message']
Y = df.drop(['message', 'genre', 'id'], axis = 1)

In [20]:
df.head(3)

,id,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,direct,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

In [22]:
Y.head(3)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26207 entries, 0 to 26206
Data columns (total 39 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      26207 non-null  int64 
 1   message                 26207 non-null  object
 2   genre                   26207 non-null  object
 3   related                 26207 non-null  int64 
 4   request                 26207 non-null  int64 
 5   offer                   26207 non-null  int64 
 6   aid_related             26207 non-null  int64 
 7   medical_help            26207 non-null  int64 
 8   medical_products        26207 non-null  int64 
 9   search_and_rescue       26207 non-null  int64 
 10  security                26207 non-null  int64 
 11  military                26207 non-null  int64 
 12  child_alone             26207 non-null  int64 
 13  water                   26207 non-null  int64 
 14  food                    26207 non-null  int64 
 15  sh

### 2. Write a tokenization function to process your text data

In [24]:
def tokenize(text):
    '''Function that takes in a text splits with white spaces and creates list of words'''
    text = text.lower() 
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    word_list = nltk.tokenize.word_tokenize(text)
    word_list = [n for n in word_list if n not in stopwords.words("english")]
    
    lemmed_list = [WordNetLemmatizer().lemmatize(n, pos='v') for n in word_list]
    
    return lemmed_list

In [25]:
test_text = 'Hello world. This is just a test'
tokenize(test_text)

['hello', 'world', 'test']

Works!

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [26]:
# Setting up the machine learning model
pipeline = Pipeline([
    ('text_pipeline', Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer())
        ])),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [34]:
# Step one: Split into train- and test data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.33)

In [38]:
# Our model to train is "pipeline"
model = pipeline.fit(X_train, y_train)

In [39]:
pred_train = model.predict(X_train)

In [40]:
pred_test = model.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [48]:
# print predictions classification_report()
for index, column in enumerate(y_test.columns):
    print('Category: {}'.format(column))
    print(classification_report(y_test[column], pred_test[:,index]))

Category: related
              precision    recall  f1-score   support

           0       0.36      0.05      0.09      2044
           1       0.76      0.97      0.85      6543
           2       0.38      0.05      0.09        62

    accuracy                           0.75      8649
   macro avg       0.50      0.36      0.34      8649
weighted avg       0.66      0.75      0.67      8649

Category: request
              precision    recall  f1-score   support

           0       0.84      0.98      0.90      7170
           1       0.44      0.06      0.10      1479

    accuracy                           0.83      8649
   macro avg       0.64      0.52      0.50      8649
weighted avg       0.77      0.83      0.77      8649

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      8609
           1       0.00      0.00      0.00        40

    accuracy                           1.00      8649
   macro avg       0.5

In [51]:
# print predictions classification_report()
for index, column in enumerate(y_train.columns):
    print('Category: {}'.format(column))
    print(classification_report(y_train[column], pred_train[:,index]))

Category: related
              precision    recall  f1-score   support

           0       1.00      0.99      0.99      4057
           1       1.00      1.00      1.00     13371
           2       1.00      0.96      0.98       130

    accuracy                           1.00     17558
   macro avg       1.00      0.98      0.99     17558
weighted avg       1.00      1.00      1.00     17558

Category: request
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     14557
           1       0.99      0.99      0.99      3001

    accuracy                           1.00     17558
   macro avg       1.00      0.99      0.99     17558
weighted avg       1.00      1.00      1.00     17558

Category: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     17479
           1       1.00      0.99      0.99        79

    accuracy                           1.00     17558
   macro avg       1.0

### 6. Improve your model
Use grid search to find better parameters.

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.